In [20]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [21]:
# ==========================================
# 1. SETUP PATHS
# ==========================================
base_osm_dir = "/Users/ruben/Desktop/Thesis/TrainingData/PH_OSM.shp"
dhs_shp_path = "/Users/ruben/Desktop/Thesis/TrainingData/PH_DHS_GPS/PHGE81FL/PHGE81FL.shp"

output_csv = "static_osm_features_detailed.csv"

In [22]:
# ==========================================
# 2. LOAD DATA
# ==========================================
print("Loading Shapefiles...")
gdf_dhs = gpd.read_file(dhs_shp_path)

# Load OSM files (Handle missing files gracefully)
try:
    gdf_roads = gpd.read_file(os.path.join(base_osm_dir, "gis_osm_roads_free_1.shp"))
    gdf_bldgs = gpd.read_file(os.path.join(base_osm_dir, "gis_osm_buildings_a_free_1.shp"))
    gdf_pois = gpd.read_file(os.path.join(base_osm_dir, "gis_osm_pois_free_1.shp"))
    print("Loaded Roads, Buildings, and POIs.")
except Exception as e:
    print(f"Error loading OSM files: {e}")
    print("Ensure you have roads, buildings, and POIs in the folder.")

Loading Shapefiles...
Loaded Roads, Buildings, and POIs.


In [23]:
# ==========================================
# 3. REPROJECT & BUFFER
# ==========================================
# Project to Meters (Philippines Zone 51N)
target_crs = "EPSG:32651"
gdf_dhs = gdf_dhs.to_crs(target_crs)
gdf_roads = gdf_roads.to_crs(target_crs)
gdf_bldgs = gdf_bldgs.to_crs(target_crs)
gdf_pois = gdf_pois.to_crs(target_crs)

# DYNAMIC BUFFERING (Tingzon Method)
# Check if DHS has 'URBAN_RURAL' column (usually 'URBAN_RURA' in shapefiles)
# U = Urban, R = Rural. If missing, default to 5km.
print("Creating Dynamic Buffers...")
def get_buffer(row):
    # Adjust column name if necessary (e.g., 'URBAN_RURA', 'TYPE')
    if 'URBAN_RURA' in row and row['URBAN_RURA'] == 'U':
        return row.geometry.buffer(2000) # 2km for Urban
    else:
        return row.geometry.buffer(5000) # 5km for Rural/Unknown

gdf_dhs['buffer_geom'] = gdf_dhs.apply(get_buffer, axis=1)

Creating Dynamic Buffers...


In [24]:
# ==========================================
# 4. FEATURE ENGINEERING LOOP
# ==========================================
print("Extracting Detailed Features (Roads, Buildings, POIs)...")
results = []

# Create Spatial Indexes for speed
road_sindex = gdf_roads.sindex
bldg_sindex = gdf_bldgs.sindex
poi_sindex = gdf_pois.sindex

for idx, row in gdf_dhs.iterrows():
    cluster_id = row['DHSCLUST']
    buffer = row['buffer_geom']
    
    # -------------------------------
    # A. ROADS (By Type)
    # -------------------------------
    possible_roads = gdf_roads.iloc[list(road_sindex.intersection(buffer.bounds))]
    precise_roads = possible_roads[possible_roads.intersects(buffer)]
    
    # Define categories based on Geofabrik 'fclass'
    road_types = {
        'Main_Roads': ['motorway', 'trunk', 'primary', 'primary_link'],
        'Secondary_Roads': ['secondary', 'tertiary'],
        'Local_Roads': ['residential', 'living_street', 'unclassified', 'service'],
        'Tracks': ['track', 'path'] # Often unpaved
    }
    
    road_stats = {}
    if len(precise_roads) > 0:
        clipped_roads = gpd.clip(precise_roads, buffer)
        road_stats['Total_Road_Length'] = clipped_roads.length.sum() / 1000.0
        
        # Calculate length per type
        for r_cat, r_classes in road_types.items():
            subset = clipped_roads[clipped_roads['fclass'].isin(r_classes)]
            road_stats[f'{r_cat}_Length'] = subset.length.sum() / 1000.0
    else:
        road_stats = {f'{k}_Length': 0 for k in road_types.keys()}
        road_stats['Total_Road_Length'] = 0

    # -------------------------------
    # B. BUILDINGS (By Type)
    # -------------------------------
    possible_bldgs = gdf_bldgs.iloc[list(bldg_sindex.intersection(buffer.bounds))]
    precise_bldgs = possible_bldgs[possible_bldgs.intersects(buffer)]
    
    bldg_stats = {'Total_Bldg_Count': len(precise_bldgs)}
    
    # Count specific types (Tingzon: residential, commercial, industrial, etc.)
    target_bldgs = ['residential', 'commercial', 'industrial', 'school', 'hospital']
    
    if len(precise_bldgs) > 0:
        counts = precise_bldgs['fclass'].value_counts()
        for t in target_bldgs:
            bldg_stats[f'Bldg_{t}_Count'] = counts.get(t, 0)
            
        # Area Stats (Total density)
        bldg_stats['Total_Bldg_Area'] = precise_bldgs.area.sum()
    else:
        for t in target_bldgs: bldg_stats[f'Bldg_{t}_Count'] = 0
        bldg_stats['Total_Bldg_Area'] = 0

    # -------------------------------
    # C. POIs (Wealth Indicators)
    # -------------------------------
    possible_pois = gdf_pois.iloc[list(poi_sindex.intersection(buffer.bounds))]
    precise_pois = possible_pois[possible_pois.intersects(buffer)]
    
    poi_stats = {'Total_POI_Count': len(precise_pois)}
    
    # Wealth-relevant POIs
    wealth_pois = ['bank', 'hotel', 'fast_food', 'convenience', 'school', 'hospital']
    
    if len(precise_pois) > 0:
        counts = precise_pois['fclass'].value_counts()
        for p in wealth_pois:
            poi_stats[f'POI_{p}_Count'] = counts.get(p, 0)
    else:
        for p in wealth_pois: poi_stats[f'POI_{p}_Count'] = 0
            
    # -------------------------------
    # COMBINE ALL
    # -------------------------------
    feature_row = {'DHSCLUST': cluster_id}
    feature_row.update(road_stats)
    feature_row.update(bldg_stats)
    feature_row.update(poi_stats)
    
    results.append(feature_row)

Extracting Detailed Features (Roads, Buildings, POIs)...


In [25]:
# ==========================================
# 5. SAVE
# ==========================================
df_final = pd.DataFrame(results)
# Force DHSCLUST to be an integer (removes the .0)
df_final['DHSCLUST'] = df_final['DHSCLUST'].astype(int)
df_final.to_csv(output_csv, index=False)
print(f"Extracted detailed OSM features for {len(df_final)} clusters.")
print(df_final.head())

Extracted detailed OSM features for 1247 clusters.
   DHSCLUST  Total_Road_Length  Main_Roads_Length  Secondary_Roads_Length  \
0         1          54.771597           1.804234                6.857690   
1         2          43.582598           5.215739                2.482889   
2         3          96.216400          13.964788               37.237548   
3         4          68.875454          13.977105                6.687322   
4         5          43.750988          10.954852                1.472004   

   Local_Roads_Length  Tracks_Length  Total_Bldg_Count  \
0           44.893212       1.216460                48   
1           29.691744       5.762369               270   
2           34.490204      10.523860               180   
3           35.270792      12.940235                53   
4           25.311176       4.468635               241   

   Bldg_residential_Count  Bldg_commercial_Count  Bldg_industrial_Count  \
0                       0                      0              